# Rod PDE System

In this notebook, we are going to build the PDESystem object related to the rod problem. Every step of the process will be sufficiently explained.

In [ ]:
# First we load the parent folder and the tools we will need

#Load of the parent folder
import sys
sys.path.insert(0, '..')

#Load of the tools we created
import AuxFuncts as AF
import System as SEQ

#Load of the sciann-related tools
import sciann as sn
from sciann import Variable
from sciann import Functional

## Definition of constants, variables, goal functions and PDESystem object.

Firstly, we will define the constants related to the whole experiment. This allows us to easily change them if needed.

Secondly, we will construct the PDESystem object. To do so, we will input the following elements:

 - The variables' names as a list of strings (order is important!)
 - The objective functions as a dictionary with format 'key':index. 
 - A dictionary holding parameters and their specs.
 - A dictionary holding the constants' names and its values.

In [ ]:
#Constant definition
TOL = 0.05 # Esta constante se usa para que los bordes tomen valores y se activen las funciones de contorno
convection = 25. # Coeficiente de pérdida de calor en los bordes y=1 e y=-1
conductivity = 1. # Capacidad conductora de calor del material de la placa
capCal = 1. # Capacidad de retención de calor del material de la placa
focoTemp = 25. # Temperatura del foco de calor
tempExt = 5. # Temperatura del exterior
tempInicial = 0. # Temperatura inicial de la placa

#PDESystem object creation
mySys = SEQ.PDESystem(['x','t'],{'u':0},paramSpecs={},
                      constants={'capCal':capCal,'TOL':TOL,'conductivity':conductivity,
                                 'convection':convection,'tempExt':tempExt})

## Definition of PDEs and the subspaces they are applied to.

The final and most important element to build the PDE System is the set of PDEs. To add them to the PDESystem object, we will create them one by one and add them using the addEquation method.

#### Heat PDE

The formula of the first PDE is: $$ C\frac{\partial u}{\partial t} -  Q \Delta u = 0,$$

where C is the heat capacity and Q the conductivity. To input this equation into the Equation object, we store it as a string, where all the variables, functions, parameters and constants should be enclosed between # symbols. In this case, the partial derivative over time is written as diff(u,x), and the diffusion component of the equation ($\Delta u$), is written as the sum of derivatives of order two over all the spatial variables (in this case, only x).

Secondly, we have to define over which subset of the problem space this equation is effective. Since the heat affects the rod over all time and space, we just call the full set with the default key 'Om'. We will see later how to adequately use the Constraints class.

Lastly, we bundle everything up and pass it to the Equation class to create an Equation object, which we add to the equation system using the addEquation method.


In [ ]:
PDE1 = '#capCal#*diff(#u#,#t#) - #conductivity# * (diff(#u#,#x#,order=2))'
valuesFunc1 = 0
constraints1 = [{
    
}, 'Om'

]

EQ1 = SEQ.Equation(constraints1,valuesFunc1,PDE1)

mySys.addEquation(EQ1)

#### Initial heat conditions.

The formula of this PDE is $$ u = 2.5*x \quad \textrm{ on } \quad X \times \lbrace t=0\rbrace $$

that is, at time $t=0$, the heat is 0 in the leftmost side of the rod, and five at the rightmost one, with the heat linearly distributed between the two points. To define this condition we plug u in the PDE side, and we plug 2.5*x in the values function side.

To build the constraint, we need to understand how the Constraint class works. To build a constraint we must define an inequation composed by three elements:
 - A values function: This is a function over the coordinates of the problem (t and x in this case), which assigns a certain value to each point depending on its coordinates.
 - A comparator: This is a string defining a comparison symbol. It can be <=, <, =, > or >.
 - A value or constant. This can only be an integer which is compared to the values function.
 
For instance, in this case we need to get all the points whose time dimension is small enough (to emulate initial conditions). To do so we define the constraint t <= TOL, where TOL is a small number.

To apply the constraint, a set operations module has been implemented inside the Equation class. This way, we can easily build complex subspaces using union (u symbol) and intersection (n symbol) operations. 

For instance if we wanted to select the initial conditions around the center of the board, we would create the previously mentioned set A, and another set B selecting the center (np.abs(x-1<=TOL)). The set would be then built as 'AnB'.

To pass the constraints on to the Equation class, we build a list composed of a dictionary containing the set objects, and a string defining the set equation.

In [ ]:
PDE2 = '#u#'
valuesFunc2 = '2.5*#x#'
constraints2 = [{
    'A':SEQ.Constraint('#t#','<=',TOL)
},'A'
]

EQ2 = SEQ.Equation(constraints2,valuesFunc2,PDE2)
mySys.addEquation(EQ2)

#### Thermal insulation.

The rod is insulated in its leftmost side. The equation defining this phenomenon is: $$ \nu \frac{\partial u}{\partial x} = 0 \quad on \quad \lbrace x=0\rbrace \times T $$

This is a vector equation, but in this case, $\nu$ along the left frontier results in $-1$ over x and $0$ over t. Therefore we get just one equation.


In [ ]:
PDE3 = '-diff(#u#,#x#)'
valuesFunc3 = 0
constraints3 = [{
    'A':SEQ.Constraint('#x#','<=',TOL)    
}, 'A'
]

EQ3 = SEQ.Equation(constraints3,valuesFunc3,PDE3)
mySys.addEquation(EQ3)

#### Constant temperature input in the rightmost side.

The rod is not insulated on the rightmost side. This is usually simulated with a convection equation similar to the previous one, but the PINN has a hard time adjusting to these conditions, so instead we input a constant temperature on said side. The equation for this condition is:

$$ u = tempExt \quad on \quad \lbrace x=2\rbrace \times T $$




In [ ]:
PDE4 = '#u#'
valuesFunc4 = tempExt
constraints4 = [{
    'A':SEQ.Constraint('#x#','>=',2-TOL)    
}, 'A'
]

EQ4 = SEQ.Equation(constraints4,valuesFunc4,PDE4)
mySys.addEquation(EQ4)

Once the Equation object is built, we can save the notebook and hand it to the experimentation worker.

Building the problem this way allows us to separate the more mathematics and physics part of the process from the more technical and programming-oriented part.